<img src="images/seattlewaterfront.jpg">

# King County House Sales Analysis
Author: Jennifer Ha
***

## Overview
This analysis project explores trends in film business to provide Microsoft with actionable recommendations as they want to enter this business as well. we will review data sets from TheNumbers.com and IMDb to determine when Microsoft should release a movie, which types of genres to consider, and whom to recruit. The methodology includes filtering, and joining and visualizing data sets to show different trends in ROI. The result shows higher ROI when Mystery, Horror, and Animation movies are released in May-July, and November-December. I recommend Microsoft to partner with lists of top 10 actors and directors in those genres, and release movies during the summer (May-July) or holiday season (Nov-Dec).

## Business Problem
Microsoft sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies.

In order to assist the stakeholders at Microsoft to make data-driven decisions for their new business, we will be reviewing and analyzing datasets to provide a viable and profitable business option. The findings and recommendations will help the stakeholders with better understanding of the movie industry, to decide when to release their movies and who to work with to drive a successful outcome.

## Data

### Data Preparation

In [106]:
#import all libraries & packages
import numpy as np
import pandas as pd
from pandas.plotting import table
import matplotlib.pyplot as plt
import matplotlib.colors as pltcol
%matplotlib inline
import seaborn as sns
import math

import statsmodels.api as sm
import statsmodels.tools.eval_measures as ev
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error, make_scorer
from sklearn.model_selection import cross_val_score
from scipy.special import boxcox, inv_boxcox


import scipy.stats as stats

#importing my personal functions 
import functions as functions

#remove scientific notation
pd.options.display.float_format = '{:,.5f}'.format

#display all the columns without truncation
pd.set_option('display.max_columns', None)

#import data into a dataframe and preview 
df = pd.read_csv('data/kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,"221,900.00000",3,1.00000,1180,5650,1.00000,nan,0.00000,3,7,1180,0.0,1955,0.00000,98178,47.51120,-122.25700,1340,5650
1,6414100192,12/9/2014,"538,000.00000",3,2.25000,2570,7242,2.00000,0.00000,0.00000,3,7,2170,400.0,1951,"1,991.00000",98125,47.72100,-122.31900,1690,7639
2,5631500400,2/25/2015,"180,000.00000",2,1.00000,770,10000,1.00000,0.00000,0.00000,3,6,770,0.0,1933,nan,98028,47.73790,-122.23300,2720,8062
3,2487200875,12/9/2014,"604,000.00000",4,3.00000,1960,5000,1.00000,0.00000,0.00000,5,7,1050,910.0,1965,0.00000,98136,47.52080,-122.39300,1360,5000
4,1954400510,2/18/2015,"510,000.00000",3,2.00000,1680,8080,1.00000,0.00000,0.00000,3,8,1680,0.0,1987,0.00000,98074,47.61680,-122.04500,1800,7503


In [107]:
#get summary of the DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

From the above summary, we see that 'date', 'yr_built', 'yr_renovated', and 'sqft_basement' have wrong data type. 'waterfront', 'view', and 'yr_renovated' also have null values that we need to take care of. Let's check for duplicates first and we will clean up the data step by step.

### Check for Duplicates

In [108]:
#check for duplicates in 'id' which should work as index
df.id.duplicated().sum()

177

In [109]:
#create a separate table to examine the duplicates
df_id_duplicates = df[df.duplicated('id')]

In [110]:
df_id_duplicates

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
94,6021501535,12/23/2014,"700,000.00000",3,1.50000,1580,5000,1.00000,0.00000,0.00000,3,8,1290,290.0,1939,0.00000,98117,47.68700,-122.38600,1570,4500
314,4139480200,12/9/2014,"1,400,000.00000",4,3.25000,4290,12103,1.00000,0.00000,3.00000,3,11,2690,1600.0,1997,0.00000,98006,47.55030,-122.10200,3860,11244
325,7520000520,3/11/2015,"240,500.00000",2,1.00000,1240,12092,1.00000,0.00000,0.00000,3,6,960,280.0,1922,"1,984.00000",98146,47.49570,-122.35200,1820,7460
346,3969300030,12/29/2014,"239,900.00000",4,1.00000,1000,7134,1.00000,0.00000,0.00000,3,6,1000,0.0,1943,nan,98178,47.48970,-122.24000,1020,7138
372,2231500030,3/24/2015,"530,000.00000",4,2.25000,2180,10754,1.00000,0.00000,0.00000,5,7,1100,1080.0,1954,0.00000,98133,47.77110,-122.34100,1810,6929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20165,7853400250,2/19/2015,"645,000.00000",4,3.50000,2910,5260,2.00000,0.00000,0.00000,3,9,2910,0.0,2012,0.00000,98065,47.51680,-121.88300,2910,5260
20597,2724049222,12/1/2014,"220,000.00000",2,2.50000,1000,1092,2.00000,0.00000,0.00000,3,7,990,10.0,2004,0.00000,98118,47.54190,-122.27100,1330,1466
20654,8564860270,3/30/2015,"502,000.00000",4,2.50000,2680,5539,2.00000,nan,0.00000,3,8,2680,0.0,2013,0.00000,98045,47.47590,-121.73400,2680,5992
20764,6300000226,5/4/2015,"380,000.00000",4,1.00000,1200,2171,1.50000,0.00000,0.00000,3,7,1200,0.0,1933,0.00000,98133,47.70760,-122.34200,1130,1598


In [111]:
#spot check data
df.loc[df['id'] == 6021501535]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
93,6021501535,7/25/2014,"430,000.00000",3,1.50000,1580,5000,1.00000,0.00000,0.00000,3,8,1290,290.0,1939,0.00000,98117,47.68700,-122.38600,1570,4500
94,6021501535,12/23/2014,"700,000.00000",3,1.50000,1580,5000,1.00000,0.00000,0.00000,3,8,1290,290.0,1939,0.00000,98117,47.68700,-122.38600,1570,4500


In [112]:
#spot check data
df.loc[df['id'] == 3969300030]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
345,3969300030,7/23/2014,"165,000.00000",4,1.00000,1000,7134,1.00000,0.00000,0.00000,3,6,1000,0.0,1943,0.00000,98178,47.48970,-122.24000,1020,7138
346,3969300030,12/29/2014,"239,900.00000",4,1.00000,1000,7134,1.00000,0.00000,0.00000,3,6,1000,0.0,1943,nan,98178,47.48970,-122.24000,1020,7138


In [113]:
#spot check data
df.loc[df['id'] == 6300000226]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
20763,6300000226,6/26/2014,"240,000.00000",4,1.00000,1200,2171,1.50000,0.00000,0.00000,3,7,1200,0.0,1933,0.00000,98133,47.70760,-122.34200,1130,1598
20764,6300000226,5/4/2015,"380,000.00000",4,1.00000,1200,2171,1.50000,0.00000,0.00000,3,7,1200,0.0,1933,0.00000,98133,47.70760,-122.34200,1130,1598


It appears that houses with multiple transactions were recorded with the same id for each transaction. I'm only going to keep the latest transaction.

In [114]:
#drop the older data
df.drop_duplicates(subset= 'id', keep= 'last', inplace= True)

In [115]:
#check if the duplicates are removed
df.id.duplicated().sum()

0

### Datetime - date

In [116]:
#convert date, yr_built, yr_renovated to datetime 
df['date'] = pd.to_datetime(df['date'])

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21420 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             21420 non-null  int64         
 1   date           21420 non-null  datetime64[ns]
 2   price          21420 non-null  float64       
 3   bedrooms       21420 non-null  int64         
 4   bathrooms      21420 non-null  float64       
 5   sqft_living    21420 non-null  int64         
 6   sqft_lot       21420 non-null  int64         
 7   floors         21420 non-null  float64       
 8   waterfront     19067 non-null  float64       
 9   view           21357 non-null  float64       
 10  condition      21420 non-null  int64         
 11  grade          21420 non-null  int64         
 12  sqft_above     21420 non-null  int64         
 13  sqft_basement  21420 non-null  object        
 14  yr_built       21420 non-null  int64         
 15  yr_renovated   1760

### Integer - sqft_basement

In [117]:
#check why sqft_basement is an object and not an integer
df.sqft_basement.value_counts()

0.0       12716
?           452
600.0       216
700.0       206
500.0       206
          ...  
2610.0        1
1284.0        1
274.0         1
1770.0        1
172.0         1
Name: sqft_basement, Length: 304, dtype: int64

In [118]:
#replace '?' in sqft_basement to 0 since they are unidentifiable
df['sqft_basement'] = df['sqft_basement'].replace('?', '0.0')

df.sqft_basement.value_counts()

0.0       13168
600.0       216
700.0       206
500.0       206
800.0       201
          ...  
2610.0        1
1284.0        1
274.0         1
1770.0        1
172.0         1
Name: sqft_basement, Length: 303, dtype: int64

### Outlier

Now let's get general summary of the data we will be working with.

In [119]:
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,"21,420.00000","21,420.00000","21,420.00000","21,420.00000","21,420.00000","21,420.00000","21,420.00000","19,067.00000","21,357.00000","21,420.00000","21,420.00000","21,420.00000","21,420.00000","17,607.00000","21,420.00000","21,420.00000","21,420.00000","21,420.00000","21,420.00000"
mean,"4,580,939,869.15761","541,861.42815",3.37395,2.11843,"2,083.13263","15,128.03800",1.49599,0.00766,0.23468,3.41078,7.66279,"1,791.17021","1,971.09300",83.89010,"98,077.87437",47.56020,-122.21378,"1,988.38408","12,775.71816"
std,"2,876,761,172.44841","367,556.93785",0.92540,0.76872,918.80841,"41,530.79684",0.54008,0.08717,0.76646,0.65004,1.17197,828.69296,29.38714,400.53447,53.47748,0.13859,0.14079,685.53706,"27,345.62187"
min,"1,000,102.00000","78,000.00000",1.00000,0.50000,370.00000,520.00000,1.00000,0.00000,0.00000,1.00000,3.00000,370.00000,"1,900.00000",0.00000,"98,001.00000",47.15590,-122.51900,399.00000,651.00000
25%,"2,123,537,386.75000","324,950.00000",3.00000,1.75000,"1,430.00000","5,040.00000",1.00000,0.00000,0.00000,3.00000,7.00000,"1,200.00000","1,952.00000",0.00000,"98,033.00000",47.47120,-122.32800,"1,490.00000","5,100.00000"
50%,"3,904,921,185.00000","450,550.00000",3.00000,2.25000,"1,920.00000","7,614.00000",1.50000,0.00000,0.00000,3.00000,7.00000,"1,560.00000","1,975.00000",0.00000,"98,065.00000",47.57210,-122.23000,"1,840.00000","7,620.00000"
75%,"7,308,900,186.25000","645,000.00000",4.00000,2.50000,"2,550.00000","10,690.50000",2.00000,0.00000,0.00000,4.00000,8.00000,"2,220.00000","1,997.00000",0.00000,"98,117.00000",47.67810,-122.12500,"2,370.00000","10,086.25000"
max,"9,900,000,190.00000","7,700,000.00000",33.00000,8.00000,"13,540.00000","1,651,359.00000",3.50000,1.00000,4.00000,5.00000,13.00000,"9,410.00000","2,015.00000","2,015.00000","98,199.00000",47.77760,-121.31500,"6,210.00000","871,200.00000"


In [120]:
# check house(s) with 33 bedrooms
df.bedrooms.value_counts()

3     9731
4     6849
2     2736
5     1586
6      265
1      191
7       38
8       13
9        6
10       3
11       1
33       1
Name: bedrooms, dtype: int64

In [121]:
#check how many transactions had 33 bedrooms
df[df['bedrooms'] >= 33]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
15856,2402100895,2014-06-25,"640,000.00000",33,1.75000,1620,6000,1.00000,0.00000,0.00000,5,7,1040,580.0,1947,0.00000,98103,47.68780,-122.33100,1330,4700


In [122]:
#remove the outlier
df.drop(index=15856, inplace= True)

### Null Values

In [123]:
#check 'waterfront'
df.waterfront.value_counts(normalize = True)

0.00000   0.99234
1.00000   0.00766
Name: waterfront, dtype: float64

In [124]:
#check 'view'
df.view.value_counts(normalize = True)

0.00000   0.90148
2.00000   0.04476
3.00000   0.02365
1.00000   0.01541
4.00000   0.01470
Name: view, dtype: float64

In [125]:
#check 'yr_renovated'
df.yr_renovated.value_counts(normalize = True)

0.00000       0.95797
2,014.00000   0.00415
2,003.00000   0.00176
2,013.00000   0.00176
2,007.00000   0.00170
                ...  
1,934.00000   0.00006
1,971.00000   0.00006
1,954.00000   0.00006
1,950.00000   0.00006
1,944.00000   0.00006
Name: yr_renovated, Length: 70, dtype: float64

Above analysis shows that:
1. 99.2% of houses in the dataseet are not near the Seattle waterfront.
2. 90.1% of houses do not have a view. 
3. 95.8% of houses have not been renovated since it was built.

While having such large amount of null values can be problematic, it makes sense in this scenario.Therefore, we will simply replace those null values with zeros. 

In [126]:
#replace null values with zeros
df.fillna(0, inplace= True)

Lastly, let's re-set the index.

In [127]:
#re-set index
df.reset_index(inplace = True, drop = True)

#double-check data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21419 entries, 0 to 21418
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             21419 non-null  int64         
 1   date           21419 non-null  datetime64[ns]
 2   price          21419 non-null  float64       
 3   bedrooms       21419 non-null  int64         
 4   bathrooms      21419 non-null  float64       
 5   sqft_living    21419 non-null  int64         
 6   sqft_lot       21419 non-null  int64         
 7   floors         21419 non-null  float64       
 8   waterfront     21419 non-null  float64       
 9   view           21419 non-null  float64       
 10  condition      21419 non-null  int64         
 11  grade          21419 non-null  int64         
 12  sqft_above     21419 non-null  int64         
 13  sqft_basement  21419 non-null  object        
 14  yr_built       21419 non-null  int64         
 15  yr_renovated   2141

## Model 1

## Model 2

## Model 3

## Conclusions
This analysis leads to three recommendations for creating movies :

**1. Release movies in May, June, July, November, or December.** Our analysis shows that summertime and holiday seasons are the best time of year to release a movie. If Microsoft wants to release a movie across the world simultaneously, I recommend launching a movie in June or July. If different timeline can be applied, release a movie in the States in May first, then internationally in June.

**2. Produce a movie in Mystery, Horror, Animation genres.** Microsoft has the capability to run movie business in many countries as it has many locations worldwide. Therefore, I recommend producing a movie in Mystery, Horror, and Animation to target audience globally.

**3. Recruit people with proven stats.** I provided lists of top 10 actors and directors in top 3 genres that generated the most profit. Microsoft should contact these individuals and try to recruit them to enter this competitive movie industry

## Next Steps
Further analyses could lead to additional insights:
* **ROI Trend in Popular Genres**: while we have identified which top 3 genres have generated the most profit, such analysis can provide whether the business is continuously growing in those genres over time.
* **Ideal Budget Range**: This modeling could predict the ideal production budget for the movies that Microsoft wants to produce.